<a href="https://colab.research.google.com/github/stader-labs/stader-protocol-simulation-notebook/blob/main/Stader_Protocol_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stader Protocol Simulator**

**Introduction**

---
This notebook contains the stader protocol simulator which can be used to understand the rewards and slashing logic. The rewards and slashing logic can be stress-tested and also tested with some expected outputs.

To use this simulator, you need to pass a test_case. Below is how to craft a test_case:

A test_case contains the following fields:


1.   Title(title): A title describing the test case. Can be any string
2.   Description(description): Description of the test case. Can be any string
3.   Initial Environment(initialEnvironment): The initial environment that the test case  starts with. This can be filled up based on the preference of the user. Furthur details of the initial environment to come below
4.   Final Environment(finalEnvironment): This is an optional field. The initialEnvironment post processing will be compared against the final environment and checked that the algorithm is performing as expected.
5.   Actions(actions): This is a list which contains the actions to be taken. 

Here is a more detailed description of some of the fields:
1.   Initial Environment(initialEnvironment): Initial environment contains the following fields:
    1. User Unprocessed Deposit(userUnprocessedDeposit)
    2. User Processed Deposit(userProcessedDeposit)
    3. User Pending Rewards(userPendingRewards)
    4. User Rewards Ratio(userRewardsRatio)
    5. User Slashing Ratio(userSlashingRatio)
    6. Global Rewards Ratio(globalRewardsRatio)
    7. Global Slashing Ratio(globalSlashingRatio)
    8. State: Accumulated Deposits(accumulatedDeposits)
    9. State: Accumulated Rewards(accumulatedRewards)

2. Actions(actions): The following actions are valid:
    1. Deposit(deposit): It takes in User(user) and Funds(funds)
    2. Epoch(epoch): It takes in Rewards(rewards) and Slashing %(slashing)
    3. Withdraw Rewards(withdraw_rewards): It takes in User (user)
    4. Undelegate (undelegate): It takes in User(user) and funds to undelegate(funds)




In [45]:
import numpy as np
import pprint
import yaml
import json

INITIAL_ENVIRONMENT = "initialEnvironment"
FINAL_ENVIRONMENT = "finalEnvironment"

USER_PROCESSED_DEPOSITS = "userProcessedDeposit"
USER_UNPROCESSED_DEPOSITS = "userUnprocessedDeposits"
USER_PENDING_REWARDS = "userPendingRewards"
USER_SLASHING_RATIOS = "userSlashingRatios"
USER_REWARDS_RATIOS = "userRewardsRatios"
USER_UNPROCESSED_UNDELEGATIONS = "userUnprocessedDelegations"
USER_PROCESSED_UNDELEGATIONS = "userProcessedDelegations"
GLOBAL_SLASHING_RATIO = "globalSlashingRatio"
GLOBAL_REWARDS_RATIO = "globalRewardsRatio"
STATE = "state"
ACCUMULATED_DEPOSITS = "accumulatedDeposits"
ACCUMULATED_REWARDS = "accumulatedRewards"

def init_env(scenario): 
    initial_env = scenario[INITIAL_ENVIRONMENT]

    # init all the user mappings
    if USER_PROCESSED_DEPOSITS not in initial_env.keys():
      initial_env[USER_PROCESSED_DEPOSITS] = {}

    if USER_UNPROCESSED_DEPOSITS not in initial_env.keys():
      initial_env[USER_UNPROCESSED_DEPOSITS] = {}

    if USER_PROCESSED_UNDELEGATIONS not in initial_env.keys():
      initial_env[USER_PROCESSED_UNDELEGATIONS] = {}

    if USER_UNPROCESSED_UNDELEGATIONS not in initial_env.keys():
      initial_env[USER_UNPROCESSED_UNDELEGATIONS] = {}

    if USER_PENDING_REWARDS not in initial_env.keys():
      initial_env[USER_PENDING_REWARDS] = {}
    
    if USER_SLASHING_RATIOS not in initial_env.keys():
      initial_env[USER_SLASHING_RATIOS] = {}

    if USER_REWARDS_RATIOS not in initial_env.keys():
      initial_env[USER_REWARDS_RATIOS] = {}

    # init global variables
    if GLOBAL_SLASHING_RATIO not in initial_env.keys():
      initial_env[GLOBAL_SLASHING_RATIO] = 1.0

    if GLOBAL_REWARDS_RATIO not in initial_env.keys():
      initial_env[GLOBAL_REWARDS_RATIO] = np.array([0.0, 0.0, 0.0])

    if STATE not in initial_env.keys():
      initial_env[STATE] = {}

    # init state variables
    if ACCUMULATED_DEPOSITS not in initial_env[STATE].keys():
      initial_env[STATE][ACCUMULATED_DEPOSITS] = 0.0

    if ACCUMULATED_REWARDS not in initial_env[STATE].keys():
      initial_env[STATE][ACCUMULATED_REWARDS] = np.array([0.0, 0.0, 0.0])

def _adjust_user_slashing(user, scenario):
    # print("Adjusting user slashing for {user}".format(user=user))

    user_slashing_ratio = scenario[INITIAL_ENVIRONMENT][USER_SLASHING_RATIOS].get(user, 1.0)
    print("User slashing ratio is {user_slashing_ratio}".format(user_slashing_ratio=user_slashing_ratio))

    if user in scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS].keys():
        scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS][user] = scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS][user] * (scenario[INITIAL_ENVIRONMENT][GLOBAL_SLASHING_RATIO] / user_slashing_ratio)

    if user in scenario[INITIAL_ENVIRONMENT][USER_REWARDS_RATIOS].keys():
        scenario[INITIAL_ENVIRONMENT][USER_REWARDS_RATIOS][user] = scenario[INITIAL_ENVIRONMENT][USER_REWARDS_RATIOS][user] * (user_slashing_ratio / scenario[INITIAL_ENVIRONMENT][GLOBAL_SLASHING_RATIO])

    scenario[INITIAL_ENVIRONMENT][USER_SLASHING_RATIOS][user] = scenario[INITIAL_ENVIRONMENT][GLOBAL_SLASHING_RATIO]


def _allocate_user_rewards(user, scenario):
    # print("Allocating user rewards for {user}".format(user=user))

    user_rewards_ratio = scenario[INITIAL_ENVIRONMENT][USER_REWARDS_RATIOS].get(user, np.array([0, 0, 0]))
    global_rewards_ratio = scenario[INITIAL_ENVIRONMENT][GLOBAL_REWARDS_RATIO]

    if all(user_rewards_ratio == global_rewards_ratio):
        # user has his/her fair share of rewards
        print(f"{user} has his/her fair share of rewards")
        return

    new_user_reward_ratio = (global_rewards_ratio - user_rewards_ratio)

    # calculate the new user_pending_rewards
    user_pending_reward = scenario[INITIAL_ENVIRONMENT][USER_PENDING_REWARDS].get(user, np.array([0.0, 0.0, 0.0]))
    user_deposits = scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS].get(user, 0)
    new_user_pending_reward = user_pending_reward + new_user_reward_ratio * user_deposits

    # update the user_pending_rewards
    if user in scenario[INITIAL_ENVIRONMENT][USER_PENDING_REWARDS].keys():
        scenario[INITIAL_ENVIRONMENT][USER_PENDING_REWARDS][user] += new_user_pending_reward
    else:
        scenario[INITIAL_ENVIRONMENT][USER_PENDING_REWARDS][user] = new_user_pending_reward

    scenario[INITIAL_ENVIRONMENT][USER_REWARDS_RATIOS][user] = global_rewards_ratio


def run_epoch(rewards, slashing_percent, scenario):
    rewards_ratio = np.array([0.0, 0.0, 0.0])

    # redeem the rewards
    if scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_DEPOSITS] != 0:
        rewards_ratio = np.array(list(map(lambda x: float(int(x) / scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_DEPOSITS]), rewards)))

        scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_REWARDS] += rewards

        slashing_percent = float(slashing_percent) / 100

    # update the global_rewards_ratio
    scenario[INITIAL_ENVIRONMENT][GLOBAL_REWARDS_RATIO] = np.add(scenario[INITIAL_ENVIRONMENT][GLOBAL_REWARDS_RATIO], np.array(rewards_ratio))

    # update slashing related stuff
    if slashing_percent != 0.0:
        print("Updating slashing stuff!")
        last_global_slashing_ratio = scenario[INITIAL_ENVIRONMENT][GLOBAL_SLASHING_RATIO]
        global_slashing_ratio = last_global_slashing_ratio * (1 - slashing_percent)
        scenario[INITIAL_ENVIRONMENT][GLOBAL_REWARDS_RATIO] = scenario[INITIAL_ENVIRONMENT][GLOBAL_REWARDS_RATIO] * (last_global_slashing_ratio / global_slashing_ratio)
        scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_DEPOSITS] = scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_DEPOSITS] * (global_slashing_ratio / last_global_slashing_ratio)
        scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_REWARDS] = scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_REWARDS] * (global_slashing_ratio / last_global_slashing_ratio)
        scenario[INITIAL_ENVIRONMENT][GLOBAL_SLASHING_RATIO] = global_slashing_ratio

    # deposit loop
    total_deposit = 0
    print("Processing unprocessed deposits")
    for user, deposit in scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_DEPOSITS].items():
        # allocate the user rewards
        _allocate_user_rewards(user, scenario)

        # adjust user slashing
        _adjust_user_slashing(user, scenario)

        # move deposits to processed deposits
        if user in scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS].keys():
            scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS][user] += deposit
        else:
            scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS][user] = deposit

        total_deposit += deposit

    # should be safe to do this in simulator
    scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_DEPOSITS] = {}
    scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_DEPOSITS] += total_deposit

    # undelegate loop
    total_undelegations = 0
    print("Processing unprocessed undelegations")
    for user, undelegation in scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_UNDELEGATIONS].items():
        _allocate_user_rewards(user, scenario)

        _adjust_user_slashing(user, scenario)

        if user in scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_UNDELEGATIONS].keys():
            scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_UNDELEGATIONS][user] += undelegation
        else:
            scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_UNDELEGATIONS][user] = undelegation

        # reduce the user deposit
        scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS][user] -= undelegation

        print("User {user} rewards are {rewards}".format(user=user, rewards=scenario[INITIAL_ENVIRONMENT][USER_PENDING_REWARDS][user]))
        # assign rewards to the user
        scenario[INITIAL_ENVIRONMENT][USER_PENDING_REWARDS][user] = np.array([0.0, 0.0, 0.0])

        total_undelegations += undelegation

    scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_UNDELEGATIONS] = {}
    scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_DEPOSITS] -= total_undelegations

def run_deposit(user, funds, scenario):
  user_unprocessed_deposits = scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_DEPOSITS]

  if user in user_unprocessed_deposits.keys():
    user_unprocessed_deposits[user] += funds
  else:
    user_unprocessed_deposits[user] = funds

def run_withdraw_rewards(user, scenario):
    if user not in scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_DEPOSITS].keys() and user not in scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS].keys():
      print(f"User {user} has not deposited to the vault")
      return

    _adjust_user_slashing(user, scenario)

    _allocate_user_rewards(user, scenario)

    user_pending_reward = scenario[INITIAL_ENVIRONMENT][USER_PENDING_REWARDS].get(user, np.array([0.0, 0.0, 0.0]))

    print("User rewards are ")
    print(user_pending_reward)
    print("")

    scenario[INITIAL_ENVIRONMENT][STATE][ACCUMULATED_REWARDS] -= user_pending_reward

    scenario[INITIAL_ENVIRONMENT][USER_PENDING_REWARDS][user] = np.array([0.0, 0.0, 0.0])

def run_undelegate(user, money, scenario):
    if user not in scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_DEPOSITS].keys() and user not in scenario[INITIAL_ENVIRONMENT][USER_PROCESSED_DEPOSITS].keys():
      print(f"User {user} has not deposited to the vault")
      return

    if user in scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_UNDELEGATIONS].keys():
        scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_UNDELEGATIONS][user] += money
    else:
        scenario[INITIAL_ENVIRONMENT][USER_UNPROCESSED_UNDELEGATIONS][user] = money

def run_scenario(scenario):
    init_env(scenario)

    pp = pprint.PrettyPrinter(depth=6)

    i = 1
    for action in scenario["actions"]:
      print("")
      if action["name"] == "epoch":
        epoch_args = action["args"]
        print("{i}: Running Epoch with rewards {rewards} and slashing {slashing}\n".format(i=i, rewards=epoch_args["rewards"], slashing=epoch_args["slashing"]))
        run_epoch(epoch_args["rewards"], epoch_args["slashing"], scenario)
      elif action["name"] == "deposit":
        deposit_args = action["args"]
        print("{i}: User {user} is depositing {funds} funds\n".format(i=i, user=deposit_args["user"], funds=deposit_args["funds"]))
        run_deposit(deposit_args["user"], deposit_args["funds"], scenario)
      elif action["name"] == "withdraw_rewards":
        withdraw_rewards_args = action["args"]
        print("{i}: User {user} is withdrawing rewards\n".format(i=i, user=withdraw_rewards_args["user"]))
        run_withdraw_rewards(withdraw_rewards_args["user"], scenario)
      elif action["name"] == "undelegate":
        undelegate_args = action["args"]
        print("{i}: User {user} is undelegating {funds} funds\n".format(i=i, user=undelegate_args["user"], funds=undelegate_args["funds"]))
        run_undelegate(undelegate_args["user"], undelegate_args["funds"], scenario)
        print("Running undelegate")

      i += 1
      # pp.pprint(scenario)
      

def display_environment(initial_environment):
    import copy

    env_to_display = copy.deepcopy(initial_environment)

    # convert all the numpy arrays to a json serializable format like list
    # USER_PENDING_REWARDS, USER_REWARDS_RATIO, GLOBAL_REWARD_RATIO, ACCUMULATED_REWARDS
    env_to_display[GLOBAL_REWARDS_RATIO] = list(env_to_display[GLOBAL_REWARDS_RATIO])
    env_to_display[STATE][ACCUMULATED_REWARDS] = list(env_to_display[STATE][ACCUMULATED_REWARDS])

    user_pending_rewards = {}
    for k, v in env_to_display[USER_PENDING_REWARDS].items():
      user_pending_rewards[k] = list(v)


    user_rewards_ratios = {}
    for k, v in env_to_display[USER_REWARDS_RATIOS].items():
      user_rewards_ratios[k] = list(v)

    env_to_display[USER_PENDING_REWARDS] = user_pending_rewards
    env_to_display[USER_REWARDS_RATIOS] = user_rewards_ratios

    print(json.dumps(env_to_display, sort_keys=False, indent=4))


def compare_output(scenario):
  if FINAL_ENVIRONMENT not in scenario.keys():
    return

  print("Comparing Initial Environment with Final Environment\n")
  message = "Initial Environment and Final Environment match!\n"

  if USER_PROCESSED_DEPOSITS in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if USER_UNPROCESSED_DEPOSITS in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if USER_PENDING_REWARDS in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if USER_SLASHING_RATIOS in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if USER_REWARDS_RATIOS in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if USER_UNPROCESSED_UNDELEGATIONS in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if USER_PROCESSED_UNDELEGATIONS in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if GLOBAL_SLASHING_RATIO in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if GLOBAL_REWARDS_RATIO in scenario[FINAL_ENVIRONMENT].keys():
    pass

  if STATE in scenario[FINAL_ENVIRONMENT].keys():
    if ACCUMULATED_DEPOSITS in scenario[FINAL_ENVIRONMENT][STATE].keys():
      pass

    if ACCUMULATED_REWARDS in scenario[FINAL_ENVIRONMENT][STATE].keys():
      pass

  print(message)

In [44]:
test_case = {
  "title": "Deposit, withdraw rewards and undelegation",
  "description": "",
  "initialEnvironment": {
    "userUnprocessedDeposits": {
      "U1": 200,
      "U2": 100
    },
    "state": {
      "accumulatedDeposits": 0
    }
  },
  "actions": [
    {
      "name": "epoch",
      "args": {
        "rewards": [0, 0, 0],
        "slashing": 0
      }
    },
    {
      "name": "epoch",
      "args": {
        "rewards": [10, 20, 30],
        "slashing": 0
      }
    },
    {
      "name": "deposit",
      "args": {
        "user": "U3",
        "funds": 100
      }
    },
    {
      "name": "epoch",
      "args": {
        "rewards": [50, 60, 70],
        "slashing": 0
      }
    },
    {
      "name": "withdraw_rewards",
      "args": {
        "user": "U2"
      }
    },
    {
      "name": "withdraw_rewards",
      "args": {
        "user": "U1"
      }
    },
    {
      "name": "epoch",
      "args": {
        "rewards": [50, 60, 70],
        "slashing": 10
      }
    },
    {
      "name": "withdraw_rewards",
      "args": {
        "user": "U3"
      }
    },
    {
      "name": "withdraw_rewards",
      "args": {
        "user": "U2"
      }
    },
    {
      "name": "withdraw_rewards",
      "args": {
        "user": "U1"
      }
    },
    {
      "name": "undelegate",
      "args": {
        "user": "U1",
        "funds": 10
      }
    },
    {
      "name": "epoch",
      "args": {
        "rewards": [10,30,50],
        "slashing": 0
      }
    },
    # {
    #   "name": "withdraw_rewards",
    #   "args": {
    #     "user": "U2"
    #   }
    # },
    # {
    #   "name": "epoch",
    #   "args": {
    #     "rewards": [20,30,40],
    #     "slashing": 0
    #   }
    # },
  ]
}

run_scenario(test_case)

compare_output(test_case)

print("")
print("The final environment is ")
display_environment(test_case[INITIAL_ENVIRONMENT])


1: Running Epoch with rewards [0, 0, 0] and slashing 0

Processing unprocessed deposits
U1 has his/her fair share of rewards
Adjusting user slashing for U1
User slashing ratio is 1.0
U2 has his/her fair share of rewards
Adjusting user slashing for U2
User slashing ratio is 1.0
Processing unprocessed undelegations
user_pending_rewards are
{}

2: Running Epoch with rewards [10, 20, 30] and slashing 0

Processing unprocessed deposits
Processing unprocessed undelegations
user_pending_rewards are
{}

3: User U3 is depositing 100 funds


4: Running Epoch with rewards [50, 60, 70] and slashing 0

Processing unprocessed deposits
user_pending_rewards are
{'U3': array([0., 0., 0.])}
Adjusting user slashing for U3
User slashing ratio is 1.0
Processing unprocessed undelegations
user_pending_rewards are
{'U3': array([0., 0., 0.])}

5: User U2 is withdrawing rewards

Adjusting user slashing for U2
User slashing ratio is 1.0
user_pending_rewards are
{'U3': array([0., 0., 0.]), 'U2': array([20.      